In [1]:
import base64
import os
from datetime import datetime, timedelta
from sre_parse import State
from urllib.parse import urlencode, urlparse, parse_qs
from click import pass_context

import requests


class OAuth():
    TOKEN_URL = "https://accounts.spotify.com/api/token"
    AUTH_URL = 'https://accounts.spotify.com/authorize'
    client_id = None
    client_secret = None
    redirect_uri = None
    # grant_type = None
    response_type = 'code'
    state = None
    scope = None

    def __init__(self, client_id, client_secret, redirect_uri):
      '''
      Initialises the client with suplpied client id and secret
      '''
      self.client_id = client_id
      self.client_secret = client_secret
      self.redirect_uri = redirect_uri

    def get_auth_data(self):
        '''
        Returns the auth data necessary for the authorisation post request in the authorisation code flow
        '''
        return {'grant_type':'authorisation_code'}

    def get_auth_url(self, state=None):
        '''
        Returns the authorisation URL to be requested
        '''
        auth_url = self.AUTH_URL
        client_id = self.client_id
        # client_secret = self.client_secret
        response_type = self.response_type
        redirect_uri = self.redirect_uri
        query = {
            'client_id':client_id, 
            'response_type': response_type,
            'redirect_uri': redirect_uri
        }
        if self.scope:
            query['scope'] = self.scope
        if state is None:
            query['state'] = state
        # OPTIONAL: if instance has show dialog option, add this to the query
        urlparams = urlencode(query)
        return f'{auth_url}?{urlparams}'


    @staticmethod
    def parse_auth_url(url):
        '''
        Parses the given Authorization URL in the Authorization Code Flow, 
        returning the state and code in positions 1 and 2 respectively
        '''
        query = urlparse(url).query
        form = parse_qs(query)
        if 'error' in form:
            raise Exception('Error authorising user')
        return tuple(form.get(param) for param in ['state', 'code'])


    def parse_response_code(self, url):
        '''
        Parses the response code for the Authorization Code Flow
        '''
        _, code = self.parse_auth_url(url)
        if code is None:
            return url
        else:
            return code


    def get_auth_response(self):
        pass


    def get_auth_headers(self):
        pass


    def get_access_token(self, code=None):
        '''
        Returns access token
        '''
        auth_url = self.AUTH_URL
        query = {
            'redirect_uri':self.redirect_uri,
            'code': code or self.get_auth_response(),
            'grant_type': 'authorization_code'
        }
        if self.scope:
            query['scope'] = self.scope
        if self.state:
            query['self'] = self.state
        
        headers = self.get_auth_headers()
        auth_response = requests.post(auth_url)

    def validate_token(self, token_info):
        pass


In [2]:
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
redirect_uri = os.getenv('REDIRECT_URI')

In [3]:
s = OAuth(client_id, client_secret, redirect_uri)

In [4]:
r = requests.get(s.get_auth_url())

In [10]:
import webbrowser
r  =webbrowser.open(s.get_auth_url())

'https://accounts.spotify.com/authorize?client_id=6e1ae3a813ca4ae2ba228993b7b3e229&response_type=code&redirect_uri=http%3A%2F%2F127.0.0.1%3A5000%2Fcallback&state=None'